In [1]:
import os
from os import getcwd
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import glob
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout,GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import skimage.io
from keras import backend as K


In [2]:
data_dir = Path('/content/SARS-CoV-2') # Dataset Location Path
test_dir = os.path.join(data_dir, 'Test')
test_cov_dir = os.path.join(test_dir, 'COVID') 
test_noncov_dir = os.path.join(test_dir, 'NON-COVID')

test_cov_fnames = os.listdir(test_cov_dir)
test_noncov_fnames = os.listdir(test_noncov_dir)

print("Test COVID Images: ", len(test_cov_fnames))
print("Test NON-COVID Images: ", len(test_noncov_fnames))


Test COVID Images:  250
Test NON-COVID Images:  246


In [3]:
testGenerator = ImageDataGenerator(rescale = 1./255).flow_from_directory(test_dir, 
                                                                         batch_size=16,
                                                                         class_mode='categorical',
                                                                         shuffle=False,
                                                                         target_size=(224,224))
                                                      

Found 496 images belonging to 2 classes.


In [4]:
def mini_block(filters, x):
    conv = tf.keras.layers.Conv2D(filters , kernel_size=(3, 3),padding = "same")(x)
    bn = tf.keras.layers.BatchNormalization()(conv)
    act = tf.keras.layers.ReLU()(bn)
    return act

def double_mini_block(filters, x):
    t = mini_block(filters, x)
    t = mini_block(filters, t)
    return t

def MFL_Block(f,inp):
    block1 = mini_block(f,inp)
    block2 = double_mini_block(f,inp)
    c1 =  concatenate([block1, block2])
    conv = tf.keras.layers.Conv2D(f , kernel_size=(1, 1))(c1)
    block3 = mini_block(f,conv)
    block4 = mini_block(f,inp)
    c2 = concatenate([block3, block4])
    conv = tf.keras.layers.Conv2D(f, kernel_size=(1, 1))(c2)
    return conv

def MFL_Net():
    inp = tf.keras.layers.Input(shape=(224,224,3))
    b1 = mini_block(16,inp)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(b1)
    p1 = MFL_Block(16,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p1)
    p2 = MFL_Block(32,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p2)
    p3 = MFL_Block(64,pool)
    pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(p3)
    p4 = MFL_Block(128,pool)
    globavg = tf.keras.layers.GlobalAveragePooling2D()(p4)
    drop = tf.keras.layers.Dropout(rate=0.5)(globavg)
    out = tf.keras.layers.Dense(2, activation='softmax')(drop)
    model = tf.keras.Model(inputs=inp, outputs=out)
    model.compile(optimizer = Adam(lr=0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
    return model


In [ ]:
model = MFL_Net()
model.load_weights('SARS-CoV-2_MFL_Net.hdf5')

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

predictions = model.predict_generator(testGenerator)

y_pred = np.argmax(predictions, axis=1)

true_classes = testGenerator.classes

class_labels = list(testGenerator.class_indices.keys())   

print(class_labels)

print(confusion_matrix(testGenerator.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

In [ ]:
from sklearn.metrics import  f1_score,roc_auc_score, accuracy_score
f1_score(testGenerator.classes, y_pred)

In [ ]:
accuracy_score(testGenerator.classes, y_pred)

In [ ]:
roc_auc_score(testGenerator.classes, y_pred)

In [ ]:
cm = confusion_matrix(testGenerator.classes, y_pred)

names = ["COVID", "NON-COVID"]

group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]

labels = [f"{v1}" for v1 in group_counts]
labels = np.asarray(labels).reshape(2,2)
sns.set(font_scale=0.9)
sns.heatmap(cm, annot=labels, fmt='', xticklabels= names, yticklabels= names, annot_kws={"size": 19}, cmap='Blues')
plt.yticks(rotation=45) 

plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title('Confusion Matrix ')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('CNN Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='best',facecolor="white")
plt.grid(b=None)

ax = plt.axes()
ax.spines['bottom'].set_color('0.5')
ax.spines['top'].set_color('0.5')
ax.spines['right'].set_color('0.5')
ax.spines['left'].set_color('0.5')
ax.set_facecolor("white")

plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='best',facecolor="white")
plt.grid(b=None)
ax = plt.axes()
ax.spines['bottom'].set_color('0.5')
ax.spines['top'].set_color('0.5')
ax.spines['right'].set_color('0.5')
ax.spines['left'].set_color('0.5')
ax.patch.set_edgecolor('black')  
ax.set_facecolor("white")

plt.show()